In [1]:
import random

# Modules to import

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores.faiss import FAISS

import os
from dotenv import load_dotenv, find_dotenv

# Get OpenAI API KEY

In [2]:
load_dotenv(find_dotenv()) # read local .env file
api = os.getenv('OPENAI_API_KEY_H')

# Split Chunk

In [6]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

docs = []
for loader in os.listdir("docs"):
    if loader is not None :
        try :
            docs.extend(PyPDFLoader("docs/"+loader).load())
        except Exception as e1 :
            print(f" [ Documents ERROR ] : {e1}")
    else : break

In [7]:
docs[:2]

[Document(page_content='1-72 \nสารบัญ \nภาพโดยรวม /คู่มืออย่างง่าย         1 \nข้อมูลทั่วไป           2 \nการล็อกและการปลดล็อก         3 \nเบาะนั่งและเข็มขัดนิรภัย          4 \nแผงหน้าป้ดและอุปกรณ์ควบคุม         5 \nการสตาร์ทและการขับขี่          6 \nการใช้อุปกรณ์อํานวยความสะดวก       7 \nเมื่อเกิดเหตุฉุกเฉิน          8 \nการดูแลรักษาสภาพรถ          9 \nการบํารุงรักษา          10 \nข้อมูลจําเพาะ           11 \n \n \n \n \n \n \n \n \n \n \n \n \n  \n \n \n \n \n  \n \n \n \n \n \n \n \n \n \n ', metadata={'source': 'docs/Text_PDF_.pdf', 'page': 0}),
 Document(page_content=' \n \n \n \nแผงหน้าป้ดและอุปกรณ์ควบคุม (ด้านคนขับ ) \nภาพโดยรวม /คู่มืออย่างง่าย 1-11 \n- สวิตช์ที่ป้ดนํ้าฝนและฉีดนํ้าล้างกระจกหน้า 5-45 \n- แผงหน้าป้ดหน้า5 -2 \n- สวิตช์ควบคุมความสว่างของแผงหน้าป้ดหน้า5 -2 \n- สวิตช์ระบบควบคุมเสถียรภาพการทรงตัว (ASC)OFF หน้า6-70 \n- สวิตช์เซ็นทรัลล็อกหน้า 3-15 \n- สวิตช์ควบคุมหน้าต่างไฟฟ้าหน้า 3-19 \n- สวิตช์ล็อกหน้า 3-20 \n- สวิตช์ไฟหน้าและไฟสูง-ตํ่าหน้า 5-40 \n- สวิตช์ระบบปรับไฟสู

# Vector Database

In [8]:
splits = r_splitter.split_documents(docs)

db = FAISS.from_documents( splits, OpenAIEmbeddings( api_key = api ) )

In [9]:
splits[random.randint(0,len(splits))]

Document(page_content='ข้อควรระวัง  \n● การเลือกตําแหน่ง  “4H” หรือ “4L” (Easy Select 4WD), ตําแหน่ง  “4HLc” หรือ  “4LLc” (Super Select \n4WD II) เพื่อขับขี่บนถนนลาดยางแห้งจะสิ้นเปลืองนํ้ามันเชื้อเพลิงมากขึ้น  และอาจเกิดเสียงรวมถึงทําให้ยางสึกหรอเร็ว \nนอกจากนี้ยังอาจทําให้อุณหภูมิของนํ้ามันเฟืองท้ายสูงกว่าปกติซึ่งจะทําให้ระบบขับเคลื่อนเสียหาย  \nยิ่งไปกว่านั้น  ระบบส่งกําลังต้องรับภาระในการขับเคลื่อนมากเกินไป  มีผลทําให้นํ้ามันรั่วซึม  เกิดความร้อนจัด หรือเกิดปัญหาร้ายแรงอื่นๆ \nได้ \n \nการขับขี่บนถนนที่หิมะอัดแน่น  \nE00646500149', metadata={'source': 'docs/Text_PDF_.pdf', 'page': 191})

# CHAT

In [10]:
# Memory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer',
)

In [11]:
model_lists = [ "gpt-3.5-turbo", 'gpt-4-1106-preview', "gpt-4", ]

llm = ChatOpenAI( api_key = api, model_name = model_lists[0], temperature = 0 )

In [34]:
# qa = RetrievalQA.from_chain_type(
#     llm,
#     retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 1}),
#     chain_type="map_reduce",
#     return_source_documents = True,
# )

In [12]:
memory.clear()

In [13]:
qa = ConversationalRetrievalChain.from_llm(
            llm = llm,
            retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
            memory = memory,
            return_source_documents = True,
            return_generated_question = True,
        )

## Get answer (Testing)

In [14]:
ran = splits[random.randint(0,len(splits))]

prompt = f"""
context : '{ran.page_content}'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate question/answer by a newline.
the question and the answer should be thai language.
"""

# Test
result = qa({"question": prompt})

print(f"context : {(ran.page_content)[:100]}\npage : {(ran.metadata)['page']+1}\n\n")

print(f"{result.keys()}\n\n")
print(f"Question :\n{result['generated_question']}\n\nAnswer :\n{result['answer']}\n\nSource :\nPage : {(result['source_documents'])[0].metadata}\nContent : {(result['source_documents'])[0].page_content}")

context : หมายเหตุ  
● หากคุณยังคงกดสวิตช์  “ASC OFF” ค้างไว้หลังจากที่ฟังก์ชันควบคุมเสถียรภาพปิดไปแล้ว  “ฟังก
page : 210


dict_keys(['question', 'chat_history', 'answer', 'source_documents', 'generated_question'])


Question :

context : 'หมายเหตุ  
● หากคุณยังคงกดสวิตช์  “ASC OFF” ค้างไว้หลังจากที่ฟังก์ชันควบคุมเสถียรภาพปิดไปแล้ว  “ฟังก์ชันป้องกันการทํางานผิดพลาด ” 
จะทํางานและเปิดฟังก์ชันควบคุมเสถียรภาพอีกครั้ง  
● แม้จะปิดฟังก์ชันควบคุมเสถียรภาพไปแล้ว  แต่ฟังก์ชันนี้อาจกลับมาทํางานได้โดยขึ้นอยู่กับการเคลื่อนไหวของรถ  
 
ฟังก์ชันควบคุมการออกตัวและการเข้าโค้ง  
E00617001048 
ฟังก์ชันควบคุมการออกตัวและการเข้าโค้งป้องกันไม่ให้ล้อหมุนมากเกินไปบนถนนลื่นและช่วยให้รถที่หยุดอยู่ออกตัวได้  
นอกจากนี้ยังให้กําลังการขับเคลื่อนที่เพียงพอและทําให้การหมุนพวงมาลัยมีประสิทธิภาพเมื่อรถเลี้ยวขณะเหยียบคันเร่ง 
 
ข้อควรระวัง'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate quest

In [15]:
((result['source_documents'])[0].metadata)['page']+1

210

In [14]:
# del result

# Prepare_dataset

In [16]:
import pandas as pd

df = pd.read_csv("rerank_dataset.csv", index_col=0)

df.tail(3)

,questions,answer,doc_page,doc_content,doc_source
48,ไฟแสดงการทำงานของโหมดการขับเคลื่อน 4 ล้อจะแสดง...,ไฟแสดงการทำงานของโหมดการขับเคลื่อน 4 ล้อจะแสดง...,206,: กะพริบ (ตัวแสดงโหมดการขับเคลื่อน4 ล้อกะพริบด...,docs/Text_PDF_.pdf
49,เมื่อขับรถอยู่บนทางลาดชันควรสตาร์ทเครื่องยนต์ใ...,"ควรสตาร์ทเครื่องยนต์ในตำแหน่ง ""P"" (จอด) เมื่อข...",184,คําเตือน ●ห้ามเลื่อนคันเกียร์ไปที่ตําแหน่ง “N”...,docs/Text_PDF_.pdf
50,ขนาดกระบอกสูบของรถคือเท่าไหร่?,ขนาดกระบอกสูบของรถคือ 86.0 มม.,355,จํานวนกระบอกสูบ : สูบเรียง 4 สูบ ปริมาตรกระบอก...,docs/Text_PDF_.pdf


In [17]:
rerank_df = {"questions":[], "answer":[], "doc_page":[], "doc_content":[], "doc_source":[]}

In [18]:
for i in range(51):
    ran = splits[random.randint(0,len(splits))]
    prompt = f"""
context : '{ran.page_content}'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate question/answer by a newline.
the question and the answer should be thai language.
"""
    try :
        result = qa({"question": prompt})
        
        rerank_df["answer"].append(result['answer'])
        rerank_df["questions"].append(result['generated_question'])
        rerank_df["doc_content"].append(ran.page_content)
        rerank_df["doc_page"].append( (ran.metadata)['page']+1 )
        rerank_df["doc_source"].append( (ran.metadata)['source'] )
        
    except :
        memory.clear()
        result = qa({"question": prompt})
 

In [19]:
r_df = pd.DataFrame.from_dict(rerank_df)

print(f"number of dataset : {len(r_df)}\n\n")
r_df.tail(3)

number of dataset : 42




,questions,answer,doc_page,doc_content,doc_source
39,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf
40,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf
41,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf


In [20]:
df = pd.concat([df, r_df], 
                  ignore_index = True)

print(f"number of dataset : {len(df)}\n\n")
display(df.tail(3))

number of dataset : 93




,questions,answer,doc_page,doc_content,doc_source
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf
92,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf


In [21]:
df.to_csv("rerank_dataset.csv")